# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment, Dataset, Model, Datastore, Environment
from azureml.data.datapath import DataPath
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig
import numpy as np
import pandas as pd
import joblib, pickle

## Dataset

### Overview
The dataset I'm using tracks car mpg as a function of several car characteristics like weight and acceleration. The task performed is regression to predict mpg as a function of these characteristics.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
subscription_id = 'f9d5a085-54dc-4215-9ba6-dad5d86e60a0'
resource_group = 'aml-quickstarts-133227'
workspace_name = 'quick-starts-ws-133227'

ws = Workspace.get(name="quick-starts-ws-133227")
exp = Experiment(workspace=ws, name="Ranga")

workspace = Workspace(subscription_id, resource_group, workspace_name)

CEdata = Dataset.get_by_name(workspace, name='CE')
CEdata.download(target_path='.', overwrite=True)
VS1data = Dataset.get_by_name(workspace, name='VS1')
VS1data.download(target_path='.', overwrite=True)
TS4data = Dataset.get_by_name(workspace, name='TS4')
TS4data.download(target_path='.', overwrite=True)
TS3data = Dataset.get_by_name(workspace, name='TS3')
TS3data.download(target_path='.', overwrite=True)



['/mnt/batch/tasks/shared/LS_root/mounts/clusters/ranga/code/Users/odl_user_133227/TS3.txt']

In [3]:
SEdata = Dataset.get_by_name(workspace, name='SE')
SEdata.download(target_path='.', overwrite=True)
PS6data = Dataset.get_by_name(workspace, name='PS6')
PS6data.download(target_path='.', overwrite=True)
PS5data = Dataset.get_by_name(workspace, name='PS5')
PS5data.download(target_path='.', overwrite=True)

FS2data = Dataset.get_by_name(workspace, name='FS2')
FS2data.download(target_path='.', overwrite=True)

EPS1data = Dataset.get_by_name(workspace, name='EPS1')
EPS1data.download(target_path='.', overwrite=True)
CPdata = Dataset.get_by_name(workspace, name='CP')
CPdata.download(target_path='.', overwrite=True)


['/mnt/batch/tasks/shared/LS_root/mounts/clusters/ranga/code/Users/odl_user_133227/CP.txt']

In [4]:
cooleff = []
coolpwr = []

ps_five = []
ps_six = []
motorpower = []

volflowtwo = []

tempthree = []
tempfour = []
effactor = []
vibes = []

In [5]:
ce = open('CE.txt','r')
for line in ce:
    cooleff.append(line.split())

In [6]:
cp = open('CP.txt','r')
for line in cp:
    coolpwr.append(line.split())

effic = open('se.txt','r')
for line in effic:
    effactor.append(line.split())
    
vibe = open('VS1.txt')
for line in vibe:
    vibes.append(line.split())

In [7]:
psfive = open('PS5.txt','r')
for line in psfive:
    ps_five.append(line.split())
pssix = open('PS6.txt','r')
for line in pssix:
    ps_six.append(line.split())

In [8]:
mp = open('EPS1.txt','r')
for line in mp:
    motorpower.append(line.split())


tthree = open('TS3.txt','r')
for line in tthree:
    tempthree.append(line.split())
tfour = open('TS4.txt','r')
for line in tfour:
    tempfour.append(line.split())

In [9]:
vftwo = open('FS2.txt','r')
for line in vftwo:
    volflowtwo.append(line.split())

In [10]:
cooleff = np.reshape(cooleff,(2205,60))
coolpwr = np.reshape(coolpwr,(2205,60))
cooleffDF = pd.DataFrame(cooleff)
coolpwrDF = pd.DataFrame(coolpwr)

In [11]:
ps_five = np.reshape(ps_five,(2205,6000))
ps_five_DF = pd.DataFrame(ps_five)
ps_six = np.reshape(ps_six,(2205,6000))
ps_six_DF = pd.DataFrame(ps_six)

In [12]:
motorpower = np.reshape(motorpower,(2205,6000))
mp_DF = pd.DataFrame(motorpower)

volflowtwo = np.reshape(volflowtwo,(2205,600))
vftwo_DF = pd.DataFrame(volflowtwo)

In [13]:
tempthree = np.reshape(tempthree,(2205,60))
tthree_DF = pd.DataFrame(tempthree)
tempfour = np.reshape(tempfour,(2205,60))
tfour_DF = pd.DataFrame(tempfour)


In [14]:
effactor = np.reshape(effactor,(2205,60))
eff_DF = pd.DataFrame(effactor)
vibes = np.reshape(vibes,(2205,60))
vibes_DF = pd.DataFrame(vibes)

In [15]:
coolpwrDF = coolpwrDF.astype(float)
coolpwrDF['Avg coolpwr'] = coolpwrDF.mean(axis=1)
coolpwrDF = pd.DataFrame(coolpwrDF.loc[:,'Avg coolpwr'])

cooleffDF = cooleffDF.astype(float)
cooleffDF['Avg cooleff'] = cooleffDF.mean(axis=1)
cooleffDF = pd.DataFrame(cooleffDF.loc[:,'Avg cooleff'])

In [16]:
ps_five_DF = ps_five_DF.astype(float)
ps_five_DF['Avg psfive'] = ps_five_DF.mean(axis=1)
ps_five_DF = pd.DataFrame(ps_five_DF.loc[:,'Avg psfive'])

ps_six_DF = ps_six_DF.astype(float)
ps_six_DF['Avg pssix'] = ps_six_DF.mean(axis=1)
ps_six_DF = pd.DataFrame(ps_six_DF.loc[:,'Avg pssix'])

In [17]:
mp_DF = mp_DF.astype(float)
mp_DF['Avg mp'] = mp_DF.mean(axis=1)
mp_DF = pd.DataFrame(mp_DF.loc[:,'Avg mp'])

vftwo_DF = vftwo_DF.astype(float)
vftwo_DF['Avg vftwo'] = vftwo_DF.mean(axis=1)
vftwo_DF = pd.DataFrame(vftwo_DF.loc[:,'Avg vftwo'])

In [18]:
eff_DF = eff_DF.astype(float)
eff_DF['Avg eff'] = eff_DF.mean(axis=1)
eff_DF = pd.DataFrame(eff_DF.loc[:,'Avg eff'])

vibes_DF = vibes_DF.astype(float)
vibes_DF['Avg vibes'] = vibes_DF.mean(axis=1)
vibes_DF = pd.DataFrame(vibes_DF.loc[:,'Avg vibes'])

In [19]:
tthree_DF = tthree_DF.astype(float)
tthree_DF['Avg tthree'] = tthree_DF.mean(axis=1)
tthree_DF = pd.DataFrame(tthree_DF.loc[:,'Avg tthree'])
tfour_DF = tfour_DF.astype(float)
tfour_DF['Avg tfour'] = tfour_DF.mean(axis=1)
tfour_DF = pd.DataFrame(tfour_DF.loc[:,'Avg tfour'])

In [20]:
features = (((((((((coolpwrDF.join(cooleffDF)).join(ps_five_DF)).join(ps_six_DF)).join(mp_DF)).join(vftwo_DF)).join(eff_DF)).join(vibes_DF)).join(tthree_DF)).join(tfour_DF))

In [21]:
profiledata = Dataset.get_by_name(workspace, name='profile')
profiledata.download(target_path='.', overwrite=True)

['/mnt/batch/tasks/shared/LS_root/mounts/clusters/ranga/code/Users/odl_user_133227/profile.txt']

In [26]:
profiles=[]
prof = open('profile.txt','r')
for line in prof:
    profiles.append(line.split())

labels = np.reshape(profiles,(2205,5))
labels_DF = pd.DataFrame(labels)
labels_DF = labels_DF.astype(float)

labels_DF.columns = ['Clr cndn','Valve cndn','Pump lkg','Hydraulic press','Stable Flag (only cat)']

In [29]:
ds = features.join(labels_DF)
ds_valvecndn = ds.drop(columns=['Clr cndn','Pump lkg','Hydraulic press','Stable Flag (only cat)'])
ds_stableflag = ds.drop(columns=['Clr cndn','Valve cndn','Pump lkg','Hydraulic press'])

Avg coolpwr               float64
Avg cooleff               float64
Avg psfive                float64
Avg pssix                 float64
Avg mp                    float64
Avg vftwo                 float64
Avg eff                   float64
Avg vibes                 float64
Avg tthree                float64
Avg tfour                 float64
Clr cndn                  float64
Valve cndn                float64
Pump lkg                  float64
Hydraulic press           float64
Stable Flag (only cat)    float64
dtype: object

In [30]:
# TODO: Put your automl config here
automl_config = AutoMLConfig(
    experiment_timeout_minutes=15,
    task='regression',
    primary_metric='r2_score',
    training_data=ds_valvecndn,
    label_column_name='Valve cndn',
    n_cross_validations=3)

In [31]:
# TODO: Submit your experiment
autoexp = Experiment(workspace=ws,name='AutoRanga')
remote_run = autoexp.submit(automl_config)
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feature handling: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary de

{'runId': 'AutoML_a2b56ecc-3710-4d64-b351-ee6241c50fa5',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2021-01-03T23:49:20.189713Z',
 'endTimeUtc': '2021-01-04T00:05:48.515095Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'r2_score',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"AutoRanga","subscription_id":"f9d5a085-54dc-4215-9ba6-dad5d86e60a0","resource_group":"aml-quickstarts-133227","workspace_name":"quick-starts-ws-133227","region":"southcentralus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"r2_score","task_type":"regression","data_script":null,"validation_size":0.0,"n_cross_validations":3,"y_min":null,"y_max":null,"num_classes":null,"featuriz

In [33]:
automl_config_flag = AutoMLConfig(
    experiment_timeout_minutes=15,
    task='classification',
    primary_metric='AUC_weighted',
    training_data=ds_stableflag,
    label_column_name='Stable Flag (only cat)',
    n_cross_validations=3)

autoexp = Experiment(workspace=ws,name='AutoRanga')
remote_run_class = autoexp.submit(automl_config_flag)
RunDetails(remote_run).show()
remote_run_class.wait_for_completion(show_output=True)



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


****************************************************************************************************

****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
****************************************************************************************************

 ITERATION   PIPELINE                                       DURATION      METRIC      BEST
         0   MaxAbsScaler LightGBM                          0:00:23       0.9955    0.9955
         1   MaxAbsScaler XGBoostClassifier                 0:00:24       0.9935    0.9955
         2   MinMaxScaler RandomForest                      0:00:32       0.9860    0.9955
         3   RobustScaler ExtremeRandomTrees                0:00:26      

{'runId': 'AutoML_e8aa9c75-70ac-4155-b703-078574a2e461',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2021-01-04T00:31:56.500514Z',
 'endTimeUtc': '2021-01-04T00:48:24.168908Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"AutoRanga","subscription_id":"f9d5a085-54dc-4215-9ba6-dad5d86e60a0","resource_group":"aml-quickstarts-133227","workspace_name":"quick-starts-ws-133227","region":"southcentralus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"AUC_weighted","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":3,"y_min":null,"y_max":null,"num_classes":2,

## Run Details

The models trained feature a variety of preprocessing methods i.e. scaling to 0-1, scaling by max values, and normalization. After this preparation, a variety of models are fit. What we find is that simple stochastic gradient descents and decision trees are largely inadequate. Random Forest uses many more trees (the 'bagging' technique) with many leaves of equal weight. Not surprisingly, adding 'boosting' to bagging through XGBoost results in improved performance since we penalize leaves that don't improve model performance.
TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [34]:
best,fitted = remote_run.get_output()
bestclass,fittedclass = remote_run_class.get_output()

In [35]:
fitted.steps[-1]

('stackensembleregressor',
 StackEnsembleRegressor(base_learners=[('14',
                                        Pipeline(memory=None,
                                                 steps=[('minmaxscaler',
                                                         MinMaxScaler(copy=True,
                                                                      feature_range=(0,
                                                                                     1))),
                                                        ('extratreesregressor',
                                                         ExtraTreesRegressor(bootstrap=False,
                                                                             ccp_alpha=0.0,
                                                                             criterion='mse',
                                                                             max_depth=None,
                                                                             ma

In [36]:
fittedclass.steps[-1]

('LightGBMClassifier',
 LightGBMClassifier(boosting_type='gbdt', class_weight=None,
                    colsample_bytree=1.0, importance_type='split',
                    learning_rate=0.1, max_depth=-1, min_child_samples=20,
                    min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
                    n_jobs=1, num_leaves=31, objective=None, random_state=None,
                    reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
                    subsample_for_bin=200000, subsample_freq=0, verbose=-10))

In [37]:
#TODO: Save the best model
regrfilename = 'best_automl_model_regr.sav'
classfilename = 'best_automl_model_class.sav'

joblib.dump(fitted,regrfilename)
joblib.dump(fittedclass,classfilename)

['best_automl_model_class.sav']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [38]:
registered_regr = Model.register(workspace=ws,model_path='./best_automl_model_class.sav',model_name='registered_regr.sav')
registered_class = Model.register(workspace=ws,model_path='./best_automl_model_regr.sav',model_name='registered_class.sav')

Registering model registered_regr.sav
Registering model registered_class.sav


In [40]:
#Create inference config
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

env = Environment.get(workspace=ws,name='AzureML-AutoML')

inferenceclass = InferenceConfig(entry_script='score_class.py',environment=env) 
inferenceregr = InferenceConfig(entry_script='score_regr.py',environment=env) 

#Deploy model and check service state
from azureml.core.webservice import LocalWebservice, AciWebservice


deployfinal = AciWebservice.deploy_configuration()

In [41]:
svc_class = Model.deploy(workspace=ws,name='deployclass',models=[registered_class],inference_config=inferenceclass,deployment_config=deployfinal)
svc_class.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [42]:
svc_regr = Model.deploy(workspace=ws,name='deployregr',models=[registered_regr],inference_config=inferenceregr,deployment_config=deployfinal)
svc_regr.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [43]:
print(svc_regr.state)

Healthy


In [44]:
print(svc_class.state)

Healthy


In [45]:
print(svc_regr.scoring_uri)

http://71fa7328-2d78-4ff9-96bb-dad118b7087e.southcentralus.azurecontainer.io/score


In [46]:
print(svc_class.scoring_uri)

http://1ce83af5-7393-4105-84f8-780ed6a51fbd.southcentralus.azurecontainer.io/score


TODO: In the cell below, send a request to the web service you deployed to test it.

In [51]:
ds.columns

Index(['Avg coolpwr', 'Avg cooleff', 'Avg psfive', 'Avg pssix', 'Avg mp',
       'Avg vftwo', 'Avg eff', 'Avg vibes', 'Avg tthree', 'Avg tfour',
       'Clr cndn', 'Valve cndn', 'Pump lkg', 'Hydraulic press',
       'Stable Flag (only cat)'],
      dtype='object')

In [50]:
features.columns

Index(['Avg coolpwr', 'Avg cooleff', 'Avg psfive', 'Avg pssix', 'Avg mp',
       'Avg vftwo', 'Avg eff', 'Avg vibes', 'Avg tthree', 'Avg tfour'],
      dtype='object')

In [48]:
import requests
import json

# URL for the web service
class_scoring_uri = 'http://1ce83af5-7393-4105-84f8-780ed6a51fbd.southcentralus.azurecontainer.io/score'
regr_scoring_uri = 'http://71fa7328-2d78-4ff9-96bb-dad118b7087e.southcentralus.azurecontainer.io/score'

data = {'data':[{
    'Avg coolpwr':1.9,
    'Avg cooleff':31.4,
    'Avg psfive':9,
    'Avg pssix':9,
    'Avg mp':2495,
    'Avg vftwo':9.6,
    'Avg eff':55.3,
    'Avg vibes': 0.6,
    'Avg tthree':47.7,
    'Avg tfour': 40.8}
]
        }

input_data = json.dumps(data)

headers = {'Content-Type': 'application/json'}


# # Make the request and display the response
valvecndion = requests.post(class_scoring_uri,input_data,headers=headers)
stableflag = requests.post(regr_scoring_uri,input_data,headers=headers)


In [49]:
print("Predicted valve condition:",valvecndion.text)

Predicted valve condition: "DataException:\n\tMessage: The number of features in [fitted data](14) does not match with those in [input data](10). Please inspect your data, and make sure that features are aligned in both the Datasets.\n\tInnerException: None\n\tErrorResponse \n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"The number of features in [fitted data](14) does not match with those in [input data](10). Please inspect your data, and make sure that features are aligned in both the Datasets.\",\n        \"target\": \"X\",\n        \"inner_error\": {\n            \"code\": \"BadData\",\n            \"inner_error\": {\n                \"code\": \"InvalidDimension\",\n                \"inner_error\": {\n                    \"code\": \"DataShapeMismatch\"\n                }\n            }\n        },\n        \"reference_code\": \"c402b6c2-3870-45a7-8745-c063bd385962\"\n    }\n}"


In [ ]:
print("Predicted flag:"stableflag.text)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
svcs.delete()